In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Model, model_from_json
from keras.layers import Dense, GlobalAveragePooling2D, Input, Rescaling
from keras.utils import image_dataset_from_directory
import os
import shutil
from tqdm import tqdm


In [1]:
img_width, img_height = 224, 224
epochs = 1
batch_size = 64

In [3]:
inputs = Input(shape=(img_width, img_height, 3))
x = Rescaling(1./255)(inputs) 
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=x)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:

src_dir = 'train'
base_dir = 'train_sorted'

cats_dir = os.path.join(base_dir, 'cats')
dogs_dir = os.path.join(base_dir, 'dogs')

os.makedirs(cats_dir, exist_ok=True)
os.makedirs(dogs_dir, exist_ok=True)

for filename in tqdm(os.listdir(src_dir)):
    if filename.startswith('cat'):
        shutil.copy(os.path.join(src_dir, filename), cats_dir)
    elif filename.startswith('dog'):
        shutil.copy(os.path.join(src_dir, filename), dogs_dir)


100%|██████████| 25000/25000 [05:42<00:00, 73.02it/s]


In [5]:
train_dataset = image_dataset_from_directory(
    'train_sorted',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.1,
    subset='training',
    seed=42,
    label_mode="categorical"
)

validation_dataset = image_dataset_from_directory(
    'train_sorted',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.1,
    subset='validation',
    seed=42,
    label_mode="categorical"
)

Found 25000 files belonging to 2 classes.
Using 22500 files for training.
Found 25000 files belonging to 2 classes.
Using 2500 files for validation.


In [6]:
model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset,
    verbose=1
)

352/352 ━━━━━━━━━━━━━━━━━━━━ 2025s 6s/step - accuracy: 0.8328 - loss: 0.3583 - val_accuracy: 0.8860 - val_loss: 0.2693


In [7]:
loss, accuracy = model.evaluate(validation_dataset)
print(f"Validation loss: {loss:.4f}")
print(f"Validation accuracy: {accuracy:.4f}")

40/40 ━━━━━━━━━━━━━━━━━━━━ 187s 5s/step - accuracy: 0.8860 - loss: 0.2693
Validation loss: 0.2693
Validation accuracy: 0.8860


In [8]:
classes = ['кот', 'собака']

In [10]:
model_json = model.to_json()
json_file = open("vgg16_cat_dogs.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("vgg16_cat_dogs.weights.h5")
json_file = open("vgg16_cat_dogs.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("vgg16_cat_dogs.weights.h5")
loaded_model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

In [11]:
img = image.load_img('tester.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
prediction = loaded_model.predict(x)
print(prediction)
print(classes[np.argmax(prediction)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step
[[0.94499266 0.05500734]]
кот
